In [41]:
import pandas as pd 
from sklearn.linear_model import LogisticRegression 
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



In [35]:
csv_list = os.listdir("data/nba_games_2021/")
cols = {'home_winpct':[], 'away_winpct':[], 'home_win':[]}
pregame_data = pd.DataFrame(cols)
for csv in csv_list:
    if (csv == "urls2021"): continue
    game = pd.read_csv(f'data/nba_games_2021/{csv}')
    home = game.columns[1]
    away = game.columns[0]
    home_record = game.loc[0][1]
    away_record = game.loc[0][0]
    home_winpct = int(home_record.split("-")[0]) / (int(home_record.split("-")[0])+int(home_record.split("-")[1]))
    away_winpct = int(away_record.split("-")[0]) / (int(away_record.split("-")[0])+int(away_record.split("-")[1]))
    
    final_score = (int(game.iloc[-2][1]), int(game.iloc[-2][0]))
    home_win = 1 if final_score[0] > final_score[1] else 0
    row = [home_winpct, away_winpct, home_win]
    pregame_data.loc[len(pregame_data.index)] = row

In [46]:

predictors = pregame_data.drop(columns=["home_win"])
response = pregame_data["home_win"]
train_data, test_data, train_target, test_target = train_test_split(
    predictors, response, test_size=0.8, random_state=999)

model = LogisticRegression() 
model.fit(train_data, train_target) 
pregame_win_probabilities = model.predict_proba(test_data)